<img style="float: left" src="images/spark.png">
<img style="float: right" src="images/surfsara.png">
<hr style="clear: both">

## 03 - Word2vec

Below are number of exercises in Python and Pyspark. Press Shift-Enter to execute the code. You can use code completion by using tab.

In this notebook we will use the subset of the 20newsgroup data created in the second notebook and train a word2vec model on the content of the posts. 
1. Loading the data
2. Creating a model using ML Pipeline
3. Creating a model using MLlib
4. Using the model to create feature vectors
5. ??? do something useful with the model

During the exercises you may want to refer to the [PySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.htmlD) for more information on possible transformations and actions.

First initialize Spark and SparkSQL.

In [ ]:
sqlCtx = SQLContext(sc)

### Loading the data

The data we need from 20newsgroups is stored in [Parquet](https://parquet.apache.org) - use SparkSQL to create a DataFrame from the stored data and print the schema.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
postsDataFrame = sqlCtx.read.parquet("data/20newsgroups.selected.parquet")
postsDataFrame.<FILL IN>

### Word2vec using the ML Pipeline API

As we are starting out with a dataframe it is very straightforward to train a word2vec model and apply it to the contents field using the [Spark ML API](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml). The Spark ML API is a set of high-level APIs built on top of DataFrames and much functionality of the MLlib packages is used.

In [ ]:
from pyspark.ml.feature import Word2Vec

w2v = Word2Vec(inputCol="content", outputCol="wvector").setSeed(123456)

# Create the model based on our "content" data
model = w2v.fit(postsDataFrame)

# Apply the model to our "content" data to create a vector for each newsgroup post in our set
result = model.transform(postsDataFrame)

Next, let's inspect a few features - note that these are of type [DenseVector](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.linalg.DenseVector) - one per post. 

In [ ]:
for feature in result.select("label", "wvector").take(3):
    print(feature)

If we take a look at the schema we can see that the Pipeline has added an extra column to our data. 

In [ ]:
result.printSchema()

You can of course use the model to find synonyms for a given term.

In [ ]:
syms = model.findSynonyms("san", 10)
syms.show()

### Word2vec using MLlib

Word2vec using MLlib is a little bit more involved. Let's do this in the form of an exercise.

Start by creating an RDD that consists of only the token arrays from the postsDataFrame. Filter out any posts of length zero and print the count and two elements.


In [ ]:
# TODO: Replace <FILL IN> with appropriate code

# Select only the token arrays with length > 0
tokenizedPosts = postsDataFrame<FILL IN>
print tokenizedPosts.take(2)
print tokenizedPosts.count()

Next, initialize a word2vec model and fit the tokens. See the [word2vec documentation](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.feature.Word2Vec) for more details. In order to be able to compare the MLlib and ML Pipelines models use the same seed as the ML Pipelines example

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
from pyspark.mllib.feature import Word2Vec

word2vec = <FILL IN>
modelML = word2vec<FILL IN>

Again, print some synonyms

In [ ]:
syms = modelML.findSynonyms("san", 10)
for s in syms:
    print(s)

It would now be tempting to use the transform function of the MLlib model to transform the words in our posts to feature vectors. Unfortunately this is not possible - see the [documentation](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.feature.Word2VecModel.transform) and note the "Local use only".

There is a way however. We can get the vectors for each word from the model and broadcast these to all executors. 

In [ ]:
modelVectors = modelML.getVectors()
vocabsize = len(modelVectors)
any_word = "san"
tmp_list = modelML.findSynonyms(any_word, vocabsize-1) 

list_words = [a[0] for a in tmp_list]
list_words.append(any_word)

nfeatures = modelML.transform(any_word).array.shape[0]
nwords = len(list_words)

print "Type of the modelVectors:", type(modelVectors)
print "Number of words in the model:", nwords
print "Number of features per word: ", nfeatures

Note that the vectors are in a Java format. Before being able to broadcast we will need to convert them to Python. Note that this operation can take a while.

In [ ]:
pyModelVectors = { w: list(modelVectors[w]) for w in modelVectors }

print modelML.transform("san")
print pyModelVectors["san"]

Broadcast the Pythonized vectors of the model so each executor has access to the data.

In [ ]:
sc.broadcast(pyModelVectors)

Finally, use the broadcast vectors to transform the newsgroupdata. A utility function has been given that, from all the word vectors in a list will construct a feature. Take 3 of the posts and compare the output vectors to the ML Pipelines output.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

import numpy as np

def transformWordList(wordList, w2Vectors, numFeatures):
    numWords = len(wordList)
    wvecs=[]
    for w in wordList:
        wvecs.append(w2Vectors.get(w, [0] * numFeatures))
    swvecs = np.sum(wvecs, axis=0)
    return [float(d)/float(numWords) for d in swvecs]

postRDD = postsDataFrame.map(lambda r: (r.id, r.label, r.content)).filter(lambda (i,l,c): len(c) > 0)
transformedPostRDD = postRDD<FILL IN>

# Print 3 posts
print <FILL IN>

### ??? do something interesting with the model.

Now it is up to you to do something interesting with the data and model. Good luck, have fun!